Imports

In [ ]:
from bs4 import BeautifulSoup, Tag
import requests
import pprint
!pip install stanza
import re

In [2]:
import pandas as pd
import stanza

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Thesis2023/philoalex-main.zip

In [ ]:
import glob
import pandas as pd
grc_files = sorted(glob.glob('/content/philoalex-main/cohn-corr/*-grc1.xml'))
grc_files


In [6]:
#use whichever language you need
#nlp_grc = stanza.Pipeline(lang='grc', processors='tokenize')
nlp = stanza.Pipeline(lang='grc', processors='tokenize')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: grc (Ancient_Greek):
| Processor | Package |
-----------------------
| tokenize  | proiel  |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!


XML Edits

In [7]:
#loading the file with beautiful soup
def open_file(x):
  philo = open( x ,encoding='utf-8', errors='ignore')
  philo_soup = BeautifulSoup(philo,'lxml')
  return philo_soup



In [8]:
#removing extra tags that we don't need for text tokenizing
def removing_tags(philo_soup):
  notes= philo_soup.find_all("note")
  for note in notes:
    note.decompose()
    #removing marked mistakes from Prof Crane's corrected file (<sic> tag)
  sic= philo_soup.find_all('sic')
  for s in sic:
    s.decompose()
    #removing page break tags
  pages = philo_soup.find_all('pb')
  for p in pages:
    p.decompose()
    #removing line break tags
  breaks = philo_soup.find_all('lb')
  for b in breaks:
    b.decompose()
  return philo_soup


In [9]:
#numbers sections based on index
#works only if sections numbers and index are aligned
#otherwise use s[id]= section, p['n]
def making_sections(philo_soup):
  sections= philo_soup.find_all('p')
  for index, s in enumerate(sections):
    s['id']= f'Section{index}'
  return philo_soup

In [13]:
def making_sent_tags(philo_soup):
  sections= philo_soup.find_all('p')
  for sec_index, sec in enumerate(sections):
    doc= nlp(sec.get_text())   #applies tokenizer
    sents= [sentence.text for sentence in doc.sentences]  #acceses tuple that stanza makes of sentences
    sec.clear()  #not sure why this needs to be in here to work but...it does
    for sent_index, sent in enumerate(sents): #enumersates the sentences for each section
      sent_tag = philo_soup.new_tag('s',id= f'section{sec_index}sent{sent_index+1}')
      sent_tag.string = sent #identifies contents of the tag
      sec.append(sent_tag) #appends contents to the correct tag
  return philo_soup


In [11]:
def write_file(philo_soup,x):
  with open(f"{x}_f", 'w') as philo_file:
    philo_file.write(str(philo_soup))


In [12]:
for x in grc_files:
  opened= open_file(x)
  no_tags= removing_tags(opened)
  made_secs= making_sections(no_tags)
  made_sent_tags= making_sent_tags(made_secs)
  written= write_file(made_sent_tags,x)


/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [14]:
corr_files = sorted(glob.glob('/content/philoalex-main/cohn-corr/*-grc1.xml_f'))
corr_files

['/content/philoalex-main/cohn-corr/corr-tlg0018.tlg001.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg002.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg003.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg004.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg005.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg006.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg007.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg008.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg009.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg010.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg011.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg012.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tlg013.opp-grc1.xml_f',
 '/content/philoalex-main/cohn-corr/corr-tlg0018.tl

In [15]:
import zipfile

In [16]:

with zipfile.ZipFile('cohn_corr_grc.zip', 'w') as f:
    for y in corr_files:
        f.write(y)

In [ ]:
#problem files
#tlg 004